In [2]:
import os
import sys
import re
import pandas as pd
import mmap
from enum import Enum
from Tables import Tables
from IPython.display import display

In [3]:
class BaseOps:
    
    @staticmethod
    def bytes2Num(barray, signed=False, order='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num, length, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    

class DataPiece(BaseOps):
    
    def __init__(self, position, length):
        self.position = position
        self.length = length
        self.end = self.position + self.length
        self.signed = False
        self.order = 'little'
        
    
    def getValue(self, buffer, seg_offset):
        return self.bytes2Num(buffer[seg_offset + self.position:
                                     seg_offset + self.end],
                              self.signed,
                              self.order)
    
    def setValue(self, buffer, seg_offset, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value
        buffer[seg_offset + self.position: 
               seg_offset + self.end] = self.num2Bytes(new_value,
                                                       self.length,
                                                       self.signed,
                                                       self.order)    
        
    def incValue(self, buffer, seg_offset, inc_value):
        self.setValue(buffer, 
                      seg_offset,
                      self.getValue(buffer, seg_offset) + inc_value)
        

class BaseTable(DataPiece, Enum):
    
    def __init__(self, *params):  # Python does not have to invoke superclass __init__()
        _len = len(params)
        self.position = params[0]
        self.length = params[1]
        self.end = self.position + self.length
        self.signed = params[2] if _len > 2 else False
        self.order = params[3] if _len > 3 else 'little'
        self.table = params[4] if _len > 4 else None
        self.optimum = params[5] if _len > 5 else None
    

    def getRepr(self, buffer, seg_offset, parent=Tables):
        _value = self.getValue(buffer, seg_offset)
        if self.table:
            try:
                return getattr(parent, self.table)(_value).name
            except ValueError:
                return '0x{0:08X}'.format(_value)               
        else:
            return str(_value)
        
    def optimize(self, buffer, seg_offset):
        if self.optimum is not None:
            self.setValue(buffer, seg_offset, self.optimum)  

In [4]:
class Segments(BaseTable):
    
    EXP = (0x0018, 4, False, 'little', None, 30000000)
    Gold = (0x001c, 4, False, 'little', None, 10000)    
    States = (0x0020, 4, False, 'little', 'State', None)
    MaxHP = (0x0024, 2, False, 'little', None, 200)
    CurrentHP = (0x0026, 2, False, 'little', None, 200)
    Reputation = (0x0044, 1, True, 'little', None, 20)
    HideInShadow = (0x0045, 1, False, 'little', None, 250)
    NatureAC = (0x0046, 2, True, 'little', None, -10)
    EffectiveAC = (0x0048, 2, True, 'little', None, -10)
    CrushAC = (0x004A, 2, True, 'little', None, -10)
    MissileAC = (0x004C, 2, True, 'little', None, -10)
    PiercingAC = (0x004E, 2, True, 'little', None, -10)
    SlashingAC = (0x0050, 2, True, 'little', None, -10)
    THAC0 = (0x0052, 1, False, 'little', None, 0)
    NumOfAttack = (0x0053, 1, False, 'little', None, 10)
    SaveVsDeath = (0x0054, 1, False, 'little', None, 0)
    SaveVsWands = (0x0055, 1, False, 'little', None, 0)
    SaveVsPolymorph = (0x0056, 1, False, 'little', None, 0)
    SaveVsBreath = (0x0057, 1, False, 'little', None, 0)
    SaveVsSpell = (0x0058, 1, False, 'little', None, 0)
    ResistFire = (0x0059, 1, False, 'little', None, 100)
    ResistCold = (0x005A, 1, False, 'little', None, 100)
    ResistElectricity = (0x005B, 1, False, 'little', None, 100)
    ResistAcid = (0x005C, 1, False, 'little', None, 100)
    ResistMagic = (0x005D, 1, False, 'little', None, 100)
    ResistMagicFire = (0x005E, 1, False, 'little', None, 100)
    ResistMagicCold = (0x005F, 1, False, 'little', None, 100)
    ResistSlashing = (0x0060, 1, False, 'little', None, 100)
    ResistCrushing = (0x0061, 1, False, 'little', None, 100)
    ResistPiercing = (0x0062, 1, False, 'little', None, 100)
    ResistMissile = (0x0063, 1, False, 'little', None, 100)
    DetectIllusion = (0x0064, 1, False, 'little', None, 250)
    SetTraps = (0x0065, 1, False, 'little', None, 250)
    Lore = (0x0066, 1, False,'little',  None, 100)
    Lockpicking = (0x0067, 1, False, 'little', None, 250)
    Stealth = (0x0068, 1,False, 'little',  None, 250)
    FindTraps = (0x0069, 1, False, 'little',None, 250)
    PickPockets = (0x006A, 1, False, 'little', None, 250)
    Fatigue = (0x006B, 1,False, 'little',  None, 0)
    Intoxication = (0x006C, 1, False, 'little', None, 0)
    Luck = (0x006D, 1, False, 'little', None,  100)
    #LSwordPro = (0x006E, 1, False, 'little', None, 5)
    #SSwordPro = (0x006F, 1, False, 'little', None, 5)
    #BowPro = (0x0070, 1, False, 'little', None, 5)
    #SpearPro = (0x0071, 1, False, 'little', None, 5)
    #BluntPro = (0x0072, 1, False, 'little', None, 5)
    #SpikedPro = (0x0073, 1, False, 'little', None, 5)
    #AxePro = (0x0074, 1, False, 'little', None, 5)
    #MissilePro = (0x0075, 1, False, 'little', None, 5)
    #UnusedPro1 = (0x0076, 1, False, 'little', None, 5)
    #UnusedPro2 = (0x0077, 1, False, 'little', None, 5)
    #UnusedPro3 = (0x0078, 1, False, 'little', None, 5)
    #UnusedPro4 = (0x0079, 1, False, 'little', None, 5)
    #UnusedPro5 = (0x007a, 1, False, 'little', None, 5)
    #UnusedPro6 = (0x007b, 1, False, 'little', None, 5)
    #UnusedPro7 = (0x007c, 1, False, 'little', None, 5)
    #NightMareMode = (0x007d, 1, False, 'little', None, None)
    #Translucency = (0x007e, 1, False, 'little', None, None)
    #MurderInc = (0x007f, 1, False, 'little', None, None)
    #ReputationJoin = (0x0080, 1, False, 'little', None, None)
    #ReputationLeave = (0x0081, 1, False, 'little', None, None)
    TurnUndead = (0x0082, 1, False, 'little', None, 100)
    Tracking = (0x0083, 1, False, 'little', None, 100)
    LvClass1 = (0x0234, 1, False, 'little', None, 0)
    LvClass2 = (0x0235, 1, False, 'little', None, 0)
    LvClass3 = (0x0236, 1, False, 'little', None, 0)
    Sex = (0x0237, 1, False, 'little', 'Gender', None)
    Strength = (0x0238, 1, False, 'little', None, 25)
    StrBonus = (0x0239, 1, False, 'little', None, 0)
    Intelligence = (0x023A, 1, False, 'little', None, 25)
    Wisdom = (0x023B, 1, False, 'little', None, 25)
    Dexterity = (0x023C, 1, False, 'little', None, 25)
    Constitution = (0x023D, 1, False, 'little', None, 25)
    Charisma = (0x023E, 1, False, 'little', None, 25)
    Morale = (0x023F, 1, False, 'little', None, 100)
    MoraleBreak = (0x0240, 1, False, 'little', None, 0)
    RacialEnemy = (0x0241, 1, False, 'little', 'Race', None)
    MoraleRecovery = (0x0242, 2, False, 'little', None, 0)
    Kit = (0x0244, 4, False, 'little', 'Kit', None)
    Race = (0x0272, 1, False, 'little', 'Race', None)
    Class = (0x0273, 1, False, 'little', 'Class', None)
    Gender = (0x0275, 1, False, 'little', 'Gender', None)
    Alignment = (0x027B, 1, False, 'little', 'Alignment', None)
    KnownSpellOffset = (0x02a0, 4, False, 'little', None, None)
    KnownSpellCount = (0x02a4, 4, False, 'little', None, None)
    MemSpellOffset = (0x02a8, 4, False, 'little', None, None)
    MemSpellCount = (0x02ac, 4, False, 'little', None, None)
    MemorizedSpellOffset = (0x02b0, 4, False, 'little', None, None)
    MemorizedSpellCount = (0x02b4, 4, False, 'little', None, None)
    ItemSlotOffset = (0x02b8, 4, False, 'little', None, None)
    ItemOffset = (0x02bc, 4, False, 'little', None, None)
    ItemCount = (0x02c0, 4, False, 'little', None, None)
    EffectOffset = (0x02c4, 4, False, 'little', None, None)
    EffectCount = (0x02c8, 4, False, 'little', None, None)
    DialogRef = (0x02cc, 8, False, 'little', None, None)

In [5]:
class BaseSeg(BaseOps):
    
    def __init__(self, filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref, count_ref):
        self.filename = filename
        self.name = name
        self.buffer = buffer
        self.parent = parent
        self.extra_internal_offset = extra_internal_offset
        self.childrens = list() 
        self.next_sibling = None
        self.offset_ref = offset_ref
        self.length_ref = length_ref
        self.count_ref = count_ref
    
    
    @property
    def _start(self):
        if self.offset_ref:
            return self.offset_ref.getValue(buffer=self.buffer, 
                                            seg_offset=self.parent.adj_offset_buffer)
        else:
            return 0       
        
    @property
    def adj_offset_segment(self):
        return self._start + self.extra_internal_offset
    
    @property
    def adj_offset_buffer(self): 
        if self.parent:
            return self.parent.adj_offset_buffer + self.adj_offset_segment
        else:
            return self.adj_offset_segment      
    
    
    @property
    def length(self):
        if self.length_ref:
            return self.length_ref.getValue(buffer=self.buffer, 
                                            seg_offset=self.parent.adj_offset_buffer)
        else:
            return None
    
    @property
    def count(self):
        if self.count_ref:
            return self.count_ref.getValue(buffer=self.buffer, 
                                           seg_offset=self.parent.adj_offset_buffer)
        else:
            return None
    
    
    def expand(self, volume):
        if self.length_ref:
            self.length_ref.incValue(buffer=self.buffer, seg_offset=self.parent.adj_offset_buffer, inc_value=volume)
        if self.next_sibling:
            self.next_sibling.move(volume)
        if self.parent:
            self.parent.expand(volume)
        
    def move(self, volume):
        self.offset_ref.incValue(buffer=self.buffer, seg_offset=self.parent.adj_offset_buffer, inc_value=volume)
        if self.next_sibling:
            self.next_sibling.move(volume)     
            
    def flush(self):
        with open(self.filename, 'wb+') as _file:
            _file.write(self.buffer)

In [6]:
class CharSeg(BaseSeg):
    
    EXTRA_INTERNAL_OFFSET = 10

    def __init__(self, filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref):
        super().__init__(filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref, None)
        self.data = pd.DataFrame()
        self.read()
        self.readAppendix()
        
    def read(self):
        self.data = pd.DataFrame()
        for _name, _item in Segments.__members__.items():
            self.data = self.data.append({'Item': _name,
                                          'File_offset': _item.position + self.adj_offset_buffer,
                                          'CRE_offset': _item.position,
                                          'Length': _item.length,
                                          'Number': _item.getValue(self.buffer, self.adj_offset_buffer),
                                          'Value': _item.getRepr(self.buffer, self.adj_offset_buffer)},
                                        ignore_index=True)
        self.data['File_offset'] = self.data['File_offset'].apply(lambda x: '0x{0:06X}h'.format(int(x)))
        self.data['CRE_offset'] = self.data['CRE_offset'].apply(lambda x: '0x{0:06X}h'.format(int(x)))
        self.data['Length'] = self.data['Length'].astype(int)
        self.data['Number'] = self.data['Number'].apply(lambda x:str(int(x)))
        self.data.set_index('Item', inplace=True)
        self.data.index.name = None
        
    def optimize(self):
        for _name, _item in Segments.__members__.items():
            _item.optimize(self.buffer, self.adj_offset_buffer)
        self.effects.optimize()
        
            
    def setValue(self, item, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value       
        getattr(Segments, item.name).setValue(self.buffer, self.adj_offset_buffer, new_value)   
        
    def readAppendix(self):
        self.knownSpell = KnownSpell(self.filename, 'KnownSpell', self.buffer, self,
                                     DataPiece(Segments.KnownSpellOffset.position, 
                                               Segments.KnownSpellOffset.length),
                                     DataPiece(Segments.KnownSpellCount.position, 
                                               Segments.KnownSpellCount.length),
                                     12)
        self.memSpell = MemSpell(self.filename, 'MemSpell', self.buffer, self,
                                 DataPiece(Segments.MemSpellOffset.position, 
                                           Segments.MemSpellOffset.length),
                                 DataPiece(Segments.MemSpellCount.position, 
                                           Segments.MemSpellCount.length),
                                 16)
        self.memorizedSpell = MemorizedSpell(self.filename, 'MemorizedSpell', self.buffer, self, 
                                             DataPiece(Segments.MemorizedSpellOffset.position, 
                                                       Segments.MemorizedSpellOffset.length),
                                             DataPiece(Segments.MemorizedSpellCount.position, 
                                                       Segments.MemorizedSpellCount.length),
                                             12)
        self.itemSlots = ItemSlots(self.filename, 'ItemSlots', self.buffer, self,
                                   DataPiece(Segments.ItemSlotOffset.position, 
                                             Segments.ItemSlotOffset.length),
                                   None,
                                   2, 80)
        self.items = Items(self.filename, 'MemorizedSpell', self.buffer, self, 
                           DataPiece(Segments.ItemOffset.position, 
                                     Segments.ItemOffset.length),
                           DataPiece(Segments.ItemCount.position, 
                                     Segments.ItemCount.length),
                           20)
        self.effects = Effects(self.filename, 'MemorizedSpell', self.buffer, self, 
                               DataPiece(Segments.EffectOffset.position, 
                                         Segments.EffectOffset.length),
                               DataPiece(Segments.EffectCount.position, 
                                         Segments.EffectCount.length),
                               264)
        self.knownSpell.next_sibling = self.memSpell
        self.memSpell.next_sibling = self.memorizedSpell
        self.memorizedSpell.next_sibling = self.itemSlots
        self.itemSlots.next_sibling = self.items
        self.items.next_sibling = self.effects

In [7]:
class AppendixSeg(BaseSeg):
    
    class Record(BaseTable):
        pass
    
    def __init__(self, filename, name, buffer, parent, offset_ref, count_ref, record_length, segment_length=None):
        super().__init__(filename, name, buffer, parent, 0, offset_ref, None, count_ref)
        self.record_length = record_length
        self.data = pd.DataFrame()
        self.records = list()
        self.read()
        
    def read(self):
        self.data = pd.DataFrame()
        self.records = list()
        if self.count:
            for _i in range(self.count):
                _start = self.adj_offset_buffer + _i * self.record_length
                _end = _start + self.record_length
                self.records.append((_start, _end))
                _data = dict([(_name, _item.getRepr(self.buffer, 
                                                    _start)) for _name, _item in self.Record.__members__.items()])
                self.data = self.data.append(_data, ignore_index=True)
                
    def delete(self, index):
        if index < self.count:
            self.buffer[self.records[index][0]: self.records[index][1]] = []
            self.read()
            if self.count:
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -1)
            self.expand(-self.record_length)
    
    def insert(self, template=0):
        if template < self.count:
            self.buffer[self.records[0][0]: self:records[0][0]] = self.buffer[self.records[index][0]: 
                                                                              self:records[index][1]]
            self.read()
            if self.count:
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
            self.expand(self.record_length) 
            
    def setValue(self, item, value, index=0):
        _start = self.records[index][0]
        if not isinstance(new_value, int):
            value = value.value
        getattr(self.Record, item.name).setValue(self.buffer, _start, value)

In [8]:
class KnownSpell(AppendixSeg):  #not done
    pass

class MemSpell(AppendixSeg):
    pass

class MemorizedSpell(AppendixSeg):
    pass

class ItemSlots(AppendixSeg):
    pass

class Items(AppendixSeg):
    pass

class Effects(AppendixSeg):
    
    class WeaponOptima(Enum):
        BastardSword = 5
        LongSword = 5
        ShortSword = 5
        Axe = 5
        TwoHandedSword = 5
        Katana = 5
        ScimitarWakizashiNinjaTo = 5
        Dagger = 5
        WarHammer = 5
        Spear = 5
        Halberd = 5
        FlailMorningstar = 5
        Mace = 5
        QuarterStaff = 5
        Crossbow = 5
        LongBow = 5
        ShortBow = 5
        Darts = 5
        Sling = 5
        #Blackjack = 5
        #Gun = 5
        #MartialArts = 5
        TwoHandedWeaponSkill = 2
        SwordandShieldSkill = 2
        SingleWeaponSkill = 2
        TwoWeaponSkill = 3
        Club = 5
        #ExtraProficiency2 = 5
        #ExtraProficiency3 = 5
        #ExtraProficiency4 = 5
        #ExtraProficiency5 = 5
        #ExtraProficiency6 = 5
        #ExtraProficiency7 = 5
        #ExtraProficiency8 = 5
        #ExtraProficiency9 = 5
        #ExtraProficiency10 = 5
        #ExtraProficiency11 = 5
        #ExtraProficiency12 = 5
        #ExtraProficiency13 = 5
        #ExtraProficiency14 = 5
        #ExtraProficiency15 = 5
        #ExtraProficiency16 = 5
        #ExtraProficiency17 = 5
        #ExtraProficiency18 = 5
        #ExtraProficiency19 = 5
        #ExtraProficiency20 = 5
    
    WEAPON_OPCODE = 0xe9
    WEAPON_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    
    class Record(BaseTable):
        OPCODE = (0x0008, 4)
        TARGET = (0x000c, 4, False, 'little', 'Target')
        POWER = (0x0010, 4)
        PARAM_1 = (0x0014, 4, False, 'little', None, 5)
        PARAM_2 = (0x0018, 4, False, 'little', 'WeaponType')
        TIME = (0x001c, 4, False, 'little', 'Time')
        
    def insert(self, template=None):
        self.read()
        if isinstance(template, int) and template < self.count:
            self.buffer[self.records[0][0]: self.records[0][0]] = self.buffer[self.records[index][0]: 
                                                                              self:records[index][1]]
        elif template is None:
            self.buffer[self.records[0][0]: self.records[0][0]] = bytearray(self.WEAPON_TEMPLATE)
        elif isinstance(template, bytearray):
            self.buffer[self.records[0][0]: self.records[0][0]] = template[:]
        if self.count:
            self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
        self.expand(self.record_length) 
        self.read()
        
    @property
    def weapon(self):
        self.read()
        return self.data[self.data['OPCODE'] == str(self.WEAPON_OPCODE)]
    
    def _getWeaponOptimum(self, code):
        _weapon = Tables.WeaponType(code).name
        return getattr(self.WeaponOptima, _weapon).value
    
    def optimize(self, more_weapon=False):
        weapons = list()
        self.read()
        for _start, _end in self.records:
            if self.Record.OPCODE.getValue(self.buffer, _start) == self.WEAPON_OPCODE:
                _code = self.Record.PARAM_2.getValue(self.buffer, _start)
                weapons.append(_code)
                self.Record.PARAM_1.setValue(self.buffer, _start, self._getWeaponOptimum(_code))
        if more_weapon:
            for _weapon, _code in Tables.WeaponType.__members__.items():
                if not _code.value in weapons:
                    _template = self.WEAPON_TEMPLATE[:]
                    self.Record.PARAM_2.setValue(_template, 0, _code)
                    self.Record.PARAM_1.setValue(_template, 0, self._getWeaponOptimum(_code)) 
                    self.insert(_template)
        self.read()

In [9]:
class SaveFile(BaseOps):
    
    
    CHARSEG_EXTRA_INTERNAL_OFFSET = 10
    HEADSEG_LENGTH = 352
    CRE_STRING = bytearray('CRE V1.0', encoding='UTF-8') + b'\xff\xff\xff\xff\xff\xff\xff\xff'
    SAVE_STRING = bytearray('BALDUR.gamGAMEV2.0', encoding='UTF-8')
    
    class Blocks(BaseTable):
        GOLD = (0x0018, 4, False, 'little', None, 1000000)
        REPUTATION = (0x0054, 4, False, 'little', None, 200)
        CHAR_STRUCT_OFFSET = (0x0020, 4)
        CHAR_STRUCT_COUNT = (0x0024, 4)
        INVENTORY_OFFSET = (0x0028, 4)
        INVENTORY_COUNT = (0x002c, 4)
        NPC_STRUCT_OFFSET = (0x0030, 4)
        NPC_STRUCT_COUNT = (0x0034, 4)
        GLOBALS_OFFSET = (0x0038, 4)
        GLOBALS_COUNT = (0x003c, 4)
        FAMILIAR_OFFSET = (0x0048, 4)
        JOURNAL_OFFSET = (0x0050, 4)
        JOURNAL_COUNT = (0x004c, 4)
        FAMILIAR_INFO_OFFSET = (0x0068, 4)
        STORED_LOCATION_OFFSET = (0x006c, 4)
        STORED_LOCATION_COUNT = (0x0070, 4)
        POCKET_PLANE_OFFSET = (0x0078, 4)
        POCKET_PLANE_COUNT = (0x007c, 4)
    
    class CharHeads(BaseTable):
        POSITION = (0x00c2, 2)
        LENGTH = (0x00c6, 2)
        NAME = (0x017e, 0)

    
    def __init__(self, filename):
        self.filename = filename
        self.pwd = os.path.dirname(self.filename)
        self.file = os.path.basename(self.filename)
        with open(filename, 'rb+') as _f:
            self.buffer = bytearray(_f.read())
        self.length = len(self.buffer)
        self.save_offset = self.searchContent(self.SAVE_STRING)[0]
        self.root = BaseSeg(self.filename, 'SaveFile', self.buffer, None, self.save_offset, None, None, None)
        self.CRE = list()
        self._data = pd.DataFrame()
        self.readCharSegs()
    
    
    @property
    def data(self):
        self._data = pd.DataFrame()
        self._data = pd.concat([_.data['Value'] for _ in self.CRE], axis=1)
        self._data.columns = [_.name for _ in self.CRE]
        return self._data[~self._data.index.str.contains('Ref|Count|Offset')]
            
        
    def searchContent(self, flag):
        _results = list()
        _l = len(flag)
        for i in range(self.length - _l):
            if self.buffer[i: i+_l] == flag:
                _results.append(i)
        return _results
    
    def readContinuousString(self, offset):
        _i = 0
        while re.match('[a-zA-Z0-9-\.]', chr(self.buffer[offset+_i])):
            _i += 1
        return ''.join([chr(_) for _ in self.buffer[offset:offset+_i]])
    
    def readCharSegs(self):
        self.CRE = list()
        for _count, _offset in enumerate(self.searchContent(self.CRE_STRING)):
            _seg_offset = self.HEADSEG_LENGTH * _count
            _name =  self.readContinuousString(_seg_offset + self.CharHeads.NAME.position)
            _offset_ref = _seg_offset + self.CharHeads.POSITION.position
            _length_ref = _seg_offset + self.CharHeads.LENGTH.position
            _new_cre = CharSeg(self.filename,
                               _name,
                               self.buffer, 
                               self.root,
                               self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                               DataPiece(_offset_ref, self.CharHeads.POSITION.length),
                               DataPiece(_length_ref, self.CharHeads.LENGTH.length))
            if _count > 0:
                self.CRE[-1].next_sibling = _new_cre
            self.CRE.append(_new_cre)
        self.linkOtherSegs()
            
    def linkOtherSegs(self):
        self.npc_struct = BaseSeg(self.filename, 'npc_struct', self.buffer, self.root, 
                                  self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                  DataPiece(self.Blocks.NPC_STRUCT_OFFSET.position,
                                            self.Blocks.NPC_STRUCT_OFFSET.length),
                                  None,
                                  DataPiece(self.Blocks.NPC_STRUCT_COUNT.position,
                                            self.Blocks.NPC_STRUCT_COUNT.length)      
                                  )
        self.global_vars = BaseSeg(self.filename, 'globals', self.buffer, self.root, 
                                   self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                   DataPiece(self.Blocks.GLOBALS_OFFSET.position,
                                             self.Blocks.GLOBALS_OFFSET.length),
                                   None,
                                   DataPiece(self.Blocks.GLOBALS_COUNT.position,
                                             self.Blocks.GLOBALS_COUNT.length)      
                                   )
        self.familiar = BaseSeg(self.filename, 'familiar', self.buffer, self.root, 
                                self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                   DataPiece(self.Blocks.FAMILIAR_OFFSET.position,
                                             self.Blocks.FAMILIAR_OFFSET.length),
                                   None,
                                   None      
                                   ) 
        self.journal = BaseSeg(self.filename, 'journal', self.buffer, self.root, 
                               self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                   DataPiece(self.Blocks.JOURNAL_OFFSET.position,
                                             self.Blocks.JOURNAL_OFFSET.length),
                                   None,
                                   DataPiece(self.Blocks.JOURNAL_COUNT.position,
                                             self.Blocks.JOURNAL_COUNT.length)     
                                   )
        self.familiar_info = BaseSeg(self.filename, 'familiar_info', self.buffer, self.root, 
                                     self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                   DataPiece(self.Blocks.FAMILIAR_INFO_OFFSET.position,
                                             self.Blocks.FAMILIAR_INFO_OFFSET.length),
                                   None,
                                   None      
                                   ) 
        self.stored = BaseSeg(self.filename, 'stored', self.buffer, self.root, 
                              self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                   DataPiece(self.Blocks.STORED_LOCATION_OFFSET.position,
                                             self.Blocks.STORED_LOCATION_OFFSET.length),
                                   None,
                                   DataPiece(self.Blocks.STORED_LOCATION_COUNT.position,
                                             self.Blocks.STORED_LOCATION_COUNT.length)     
                                   )        
        self.pocket = BaseSeg(self.filename, 'pocket', self.buffer, self.root, 
                              self.CHARSEG_EXTRA_INTERNAL_OFFSET,
                                   DataPiece(self.Blocks.POCKET_PLANE_OFFSET.position,
                                             self.Blocks.POCKET_PLANE_OFFSET.length),
                                   None,
                                   DataPiece(self.Blocks.POCKET_PLANE_COUNT.position,
                                             self.Blocks.POCKET_PLANE_COUNT.length)     
                                   )
        self.CRE[-1].next_sibling = self.npc_struct
        self.npc_struct.next_sibling = self.global_vars
        self.global_vars.next_sibling = self.familiar
        self.familiar.next_sibling = self.journal
        self.journal.next_sibling = self.familiar_info
        self.familiar_info.next_sibling = self.stored
        self.stored.next_sibling = self.pocket
        
            
    def optimize(self):
        for _item in [self.Blocks.GOLD, self.Blocks.REPUTATION]:
            _item.optimize(self.buffer, self.save_offset + self.CHARSEG_EXTRA_INTERNAL_OFFSET)
        for _char in self.CRE:
            _char.optimize()
            
    def flush(self):
        with open(self.filename, 'wb+') as _file:
            _file.write(self.buffer)

In [10]:
t1 = SaveFile(r'000000001-Quick-Save-3.bg1bpsave')

In [11]:
t1.optimize()

In [31]:
t1.CRE[0].setValue(Segments.LvClass1, 9)
t1.CRE[0].setValue(Segments.LvClass2, 11)
t1.CRE[1].setValue(Segments.LvClass1, 9)
t1.CRE[1].setValue(Segments.LvClass2, 9)
t1.CRE[2].setValue(Segments.LvClass1, 8)
t1.CRE[2].setValue(Segments.LvClass2, 9)
t1.CRE[2].setValue(Segments.LvClass3, 10)
t1.CRE[3].setValue(Segments.LvClass2, 8)
t1.CRE[3].setValue(Segments.LvClass1, 9)
t1.CRE[3].setValue(Segments.LvClass3, 10)

In [16]:
t1.CRE[0].setValue(Segments.LvClass1, 0)
t1.CRE[0].setValue(Segments.LvClass2, 0)
t1.CRE[1].setValue(Segments.LvClass1, 0)
t1.CRE[1].setValue(Segments.LvClass2, 0)
t1.CRE[2].setValue(Segments.LvClass1, 0)
t1.CRE[2].setValue(Segments.LvClass2, 0)
t1.CRE[2].setValue(Segments.LvClass3, 0)
t1.CRE[3].setValue(Segments.LvClass2, 0)
t1.CRE[3].setValue(Segments.LvClass1, 0)
t1.CRE[3].setValue(Segments.LvClass3, 0)

In [17]:
t1.flush()